In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import arlpy.plot as aplt
import matplotlib.pyplot as plt
import matplotlib.style as style
from sklearn.utils import shuffle
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [ ]:
style.use('seaborn-poster')
style.use('seaborn-whitegrid')

In [ ]:
# set random seed for random number generator
np.random.seed(30)

In [ ]:
# read the data from the csv file into a dataframe
dataset = pd.read_csv('training_data/exdfp_renin.csv')
# print(dataset.shape)

In [ ]:
# pre-process the dataset to keep only the active compounds
d = []
mdataset = dataset
for i in range(dataset.shape[0]):
    if dataset['IC50'][i] <= 50000:
        continue
    else:
        d.append(i)
d.append(0)
mdataset = dataset.drop(dataset.index[d])
print(mdataset.shape)

In [ ]:
# visualize the pIC50 values in the dataset
plt.plot(-1 * np.log10(1e-9*np.asarray(mdataset['IC50'])), 'b--', linewidth=2)
plt.xlabel('Training example number')
plt.ylabel('pIC50')

In [ ]:
# number of features as input dimension
input_dim = mdataset.shape[1]-2

In [ ]:
# input matrix to neural network of size (no. of training data X no. of features)
X = np.asarray(mdataset[mdataset.columns[1:(mdataset.shape[1]-1)]])

# only needed when individual test example before shuffle are to be verified for pIC50 values
# note the corresponding number in the dataframe or excel file
Q9 = X[9,:]
Q9 = Q9.reshape(Q9.shape[0],1)

Q15 = X[15,:]
Q15 = Q15.reshape(Q15.shape[0],1)

Q20 = X[20,:]
Q20 = Q20.reshape(Q20.shape[0],1)

Q30 = X[30,:]
Q30 = Q30.reshape(Q30.shape[0],1)

Q50 = X[50,:]
Q50 = Q50.reshape(Q50.shape[0],1)

Q100 = X[100,:]
Q100 = Q100.reshape(Q100.shape[0],1)

Q200 = X[200,:]
Q200 = Q200.reshape(Q200.shape[0],1)

Q300 = X[300,:]
Q300 = Q300.reshape(Q300.shape[0],1)

Q500 = X[500,:]
Q500 = Q500.reshape(Q500.shape[0],1)

Q800 = X[800,:]
Q800 = Q800.reshape(Q800.shape[0],1)

Q900 = X[900,:]
Q900 = Q900.reshape(Q900.shape[0],1)

Q1000 = X[1000,:]
Q1000 = Q1000.reshape(Q1000.shape[0],1)

Q1200 = X[1200,:]
Q1200 = Q1200.reshape(Q1200.shape[0],1)

Q1300 = X[1300,:]
Q1300 = Q1300.reshape(Q1300.shape[0],1)

Q1400 = X[1400,:]
Q1400 = Q1400.reshape(Q1400.shape[0],1)

Q1500 = X[1500,:]
Q1500 = Q1500.reshape(Q1500.shape[0],1)

Q1600 = X[1600,:]
Q1600 = Q1600.reshape(Q1600.shape[0],1)

Q1700 = X[1700,:]
Q1700 = Q1700.reshape(Q1700.shape[0],1)

Q1800 = X[1800,:]
Q1800 = Q1800.reshape(Q1800.shape[0],1)

Q1900 = X[1900,:]
Q1900 = Q1900.reshape(Q1900.shape[0],1)

Q2000 = X[2000,:]
Q2000 = Q2000.reshape(Q2000.shape[0],1)

Q2100 = X[2100,:]
Q2100 = Q2100.reshape(Q2100.shape[0],1)

Q2200 = X[2200,:]
Q2200 = Q2200.reshape(Q2200.shape[0],1)

Q2300 = X[2300,:]
Q2300 = Q2300.reshape(Q2300.shape[0],1)

Q2400 = X[2400,:]
Q2400 = Q2400.reshape(Q2400.shape[0],1)

Q2500 = X[2500,:]
Q2500 = Q2500.reshape(Q2500.shape[0],1)


# true output pIC50 values 
Y = -1 * np.log10(1e-9*np.asarray(mdataset['IC50']))

# shuffle and reshape the data
Z = np.zeros((X.shape[0],X.shape[1]+1))
Z[:,:-1] = X
Z[:,-1] = Y
Z = shuffle(Z)
X = Z[:,:-1]
Y = Z[:,-1]
Y = Y.reshape(mdataset.shape[0],1)

In [ ]:
# split the data into training and test set
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.25)

# normalize and scale both training and test inputs
sc = StandardScaler()
sc.fit(X_train)
X_train = sc.transform(X_train)
sc.fit(X_test)
X_test = sc.transform(X_test)

In [ ]:
# define a new metric function for computing coefficient of determination (r-square)
def coeff_determination(y_true, y_pred):
    SS_res =  tf.keras.backend.sum(tf.keras.backend.square( y_true-y_pred )) 
    SS_tot = tf.keras.backend.sum(tf.keras.backend.square( y_true - tf.keras.backend.mean(y_true) ) ) 
    return ( 1 - SS_res/(SS_tot + tf.keras.backend.epsilon()) )

In [ ]:
# code for hyperparameter tuning
# NOTE: DO NOT run this code unless you want to tune the parameters
trloss = []
teloss = []
trrsqaure = []
tersqaure = []
for i in range(10, 201, 1):
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Dense(2, input_dim=input_dim, kernel_initializer='normal', activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)))
    model.add(tf.keras.layers.Dense(8, activation='relu', kernel_initializer='normal', kernel_regularizer=tf.keras.regularizers.l2(0.001)))
    model.add(tf.keras.layers.Dense(1, kernel_initializer='normal', activation='linear'))
    model.compile(loss='mean_squared_error', optimizer='adam', metrics=[coeff_determination])
    model.fit(X_train, y_train, epochs=i)
    score_train = model.evaluate(X_train, y_train)
    score_test = model.evaluate(X_test, y_test)
    trloss.append(score_train[0])
    trrsqaure.append(score_train[1])
    teloss.append(score_test[0])
    tersqaure.append(score_test[1])

x_axis = np.arange(10, 201, 1)
aplt.plot(x_axis, trloss, color='red', marker='*', legend='Training loss', ylim=[-1,2], hold=True)
aplt.plot(x_axis, trrsqaure, color='red', marker='o', legend='Training rsquare', hold=True)
aplt.plot(x_axis, teloss, color='blue', marker='*', legend='Test loss', hold=True)
aplt.plot(x_axis, tersqaure, color='blue', legend='Test rsquare', marker='o')

In [ ]:
# sequential neural network model
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(1024, input_dim=input_dim, kernel_initializer='normal', activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)))
model.add(tf.keras.layers.Dense(200, activation='relu', kernel_initializer='normal', kernel_regularizer=tf.keras.regularizers.l2(0.001)))
model.add(tf.keras.layers.Dense(1, kernel_initializer='normal', activation='linear'))
model.compile(loss='mean_squared_error', optimizer='adam', metrics=[coeff_determination])
model.fit(X_train, y_train, epochs=45)

In [ ]:
# evaluate the model on training data
scores = model.evaluate(X_train, y_train)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

# evaluate the model on test data
scores = model.evaluate(X_test, y_test)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

In [ ]:
# validate the model using test data 
y_pred = model.predict(X_test)

In [ ]:
# only needed when individual test example before shuffle are to be verified for pIC50 values
y_pred_Q9 = model.predict(Q9.transpose())
y_pred_Q15 = model.predict(Q15.transpose())
y_pred_Q20 = model.predict(Q20.transpose())
y_pred_Q30 = model.predict(Q30.transpose())
y_pred_Q50 = model.predict(Q50.transpose())
y_pred_Q100 = model.predict(Q100.transpose())
y_pred_Q200 = model.predict(Q200.transpose())
y_pred_Q300 = model.predict(Q300.transpose())
y_pred_Q500 = model.predict(Q500.transpose())
y_pred_Q800 = model.predict(Q800.transpose())
y_pred_Q900 = model.predict(Q900.transpose())
y_pred_Q1000 = model.predict(Q1000.transpose())
y_pred_Q1200 = model.predict(Q1200.transpose())
y_pred_Q1300 = model.predict(Q1300.transpose())
y_pred_Q1400 = model.predict(Q1400.transpose())
y_pred_Q1500 = model.predict(Q1500.transpose())
y_pred_Q1600 = model.predict(Q1600.transpose())
y_pred_Q1700 = model.predict(Q1700.transpose())
y_pred_Q1800 = model.predict(Q1800.transpose())
y_pred_Q1900 = model.predict(Q1900.transpose())
y_pred_Q2000 = model.predict(Q2000.transpose())
y_pred_Q2100 = model.predict(Q2100.transpose())
y_pred_Q2200 = model.predict(Q2200.transpose())
y_pred_Q2300 = model.predict(Q2300.transpose())
y_pred_Q2400 = model.predict(Q2400.transpose())
y_pred_Q2500 = model.predict(Q2500.transpose())
# y_pred_Q9_ic50 = np.power(10, -y_pred_Q9.squeeze())*1e9
# y_pred_Q15_ic50 = np.power(10, -y_pred_Q15.squeeze())*1e9
# y_pred_Q20_ic50 = np.power(10, -y_pred_Q20.squeeze())*1e9
# y_pred_Q30_ic50 = np.power(10, -y_pred_Q30.squeeze())*1e9
# y_pred_Q50_ic50 = np.power(10, -y_pred_Q50.squeeze())*1e9
# y_pred_Q100_ic50 = np.power(10, -y_pred_Q100.squeeze())*1e9
# y_pred_Q200_ic50 = np.power(10, -y_pred_Q200.squeeze())*1e9
# y_pred_Q300_ic50 = np.power(10, -y_pred_Q300.squeeze())*1e9
# y_pred_Q500_ic50 = np.power(10, -y_pred_Q500.squeeze())*1e9
# y_pred_Q800_ic50 = np.power(10, -y_pred_Q800.squeeze())*1e9
# y_pred_Q900_ic50 = np.power(10, -y_pred_Q900.squeeze())*1e9
# y_pred_Q1000_ic50 = np.power(10, -y_pred_Q1000.squeeze())*1e9
# y_pred_Q1200_ic50 = np.power(10, -y_pred_Q1200.squeeze())*1e9
# y_pred_Q1300_ic50 = np.power(10, -y_pred_Q1300.squeeze())*1e9
# y_pred_Q1400_ic50 = np.power(10, -y_pred_Q1400.squeeze())*1e9
# y_pred_Q1500_ic50 = np.power(10, -y_pred_Q1500.squeeze())*1e9
# y_pred_Q1600_ic50 = np.power(10, -y_pred_Q1600.squeeze())*1e9
# y_pred_Q1700_ic50 = np.power(10, -y_pred_Q1700.squeeze())*1e9
# y_pred_Q1800_ic50 = np.power(10, -y_pred_Q1800.squeeze())*1e9
# y_pred_Q1900_ic50 = np.power(10, -y_pred_Q1900.squeeze())*1e9
# y_pred_Q2000_ic50 = np.power(10, -y_pred_Q2000.squeeze())*1e9
# y_pred_Q2100_ic50 = np.power(10, -y_pred_Q2100.squeeze())*1e9
# y_pred_Q2200_ic50 = np.power(10, -y_pred_Q2200.squeeze())*1e9
# y_pred_Q2300_ic50 = np.power(10, -y_pred_Q2300.squeeze())*1e9
# y_pred_Q2400_ic50 = np.power(10, -y_pred_Q2400.squeeze())*1e9
# y_pred_Q2500_ic50 = np.power(10, -y_pred_Q2500.squeeze())*1e9

In [ ]:
y_pred_Q9

In [ ]:
y_pred_Q9

In [ ]:
y_pred_Q15

In [ ]:
y_pred_Q20

In [ ]:
y_pred_Q30

In [ ]:
y_pred_Q50

In [ ]:
y_pred_Q100

In [ ]:
y_pred_Q200

In [ ]:
y_pred_Q300

In [ ]:
y_pred_Q500

In [ ]:
y_pred_Q800

In [ ]:
y_pred_Q900

In [ ]:
y_pred_Q1000

In [ ]:
y_pred_Q1200

In [ ]:
y_pred_Q1300

In [ ]:
y_pred_Q1400

In [ ]:
y_pred_Q1500

In [ ]:
y_pred_Q1600

In [ ]:
y_pred_Q1700

In [ ]:
y_pred_Q1800

In [ ]:
y_pred_Q1900

In [ ]:
y_pred_Q2000

In [ ]:
y_pred_Q2100

In [ ]:
y_pred_Q2200

In [ ]:
y_pred_Q2300

In [ ]:
y_pred_Q2400

In [ ]:
y_pred_Q2500

In [ ]:
# residual plot
residues = y_test - y_pred
plt.plot(np.squeeze(residues), 'b-o', linewidth=0.5, markersize=6)
plt.xlabel('Test example number')
plt.ylabel('Residue')

In [ ]:
# model prediction vs truth
plt.figure(1, figsize=(25,10))
plt.plot(np.squeeze(y_pred), 'b--*', linewidth=0.8, markersize=5, label='Predicted pIC50')
plt.plot(np.squeeze(y_test), 'r--o', linewidth=0.8, markersize=5, label='Experimental pIC50')
plt.legend(loc='best')
plt.xlabel('Test example number')
plt.ylabel('pIC50')
# plt.savefig('fig-r1.png')

In [ ]:
# model prediction vs truth (zoomed in)
plt.figure(1, figsize=(25,10))
plt.plot(np.squeeze(y_pred[:100]), 'b--*', linewidth=0.8, markersize=5, label='Predicted pIC50')
plt.plot(np.squeeze(y_test[:100]), 'r--o', linewidth=0.8, markersize=5, label='Experimental pIC50')
plt.legend(loc='best')
plt.xlabel('Test example number')
plt.ylabel('pIC50')
# plt.savefig('fig-r2.png')

In [ ]:
# scatter plot to visulaize the model output vs true output for test
plt.scatter(np.squeeze(y_pred), np.squeeze(y_test), alpha=0.8, c='blue', s=20)
plt.plot([0,12], [0,12], c='red', linewidth=1)
plt.xlabel('Experimental pIC50')
plt.ylabel('Predicted pIC50')
# plt.savefig('fig-r3.png')

In [ ]:
# validate the model using training data 
y_pred = model.predict(X_train)

In [ ]:
# scatter plot to visulaize the model output vs true output for training 
plt.scatter(np.squeeze(y_pred), np.squeeze(y_train), alpha=0.8, c='blue', s=20)
plt.plot([0,12], [0,12], c='red', linewidth=1)
plt.xlabel('Experimental pIC50')
plt.ylabel('Predicted pIC50')
# plt.savefig('fig-r4.png')

In [ ]:
# read the unknown data set from csv file to dataframe
udataset = pd.read_csv('library_to_screen/exdfp_maybridge.csv')

In [ ]:
# curate the unknown dataset to remove duplicates
cudataset = udataset
d =[]
for i in range(udataset.shape[0]-1):
    if udataset.loc[i]['Name']==udataset.loc[i+1]['Name']:
        d.append(i+1)
cudataset = udataset.drop(udataset.index[d])
print(cudataset.shape)

In [ ]:
cudataset.to_csv('library_to_screen/exdfp_maybridge_wo_duplicates.csv', sep='\t')

In [ ]:
# read the name of compounds that were predicted to be active by DNN classifier
hits = pd.read_csv('output_hits.csv')
print(hits.shape)

In [ ]:
# create a dataset with only the actives predicted by DNN classifier
hitdataset = cudataset
d = []
for i in range(cudataset.shape[0]):
    if cudataset.iloc[i]['Name'] in list(hits['Name']):
        continue
    else:
        d.append(i)
hitdataset = cudataset.drop(cudataset.index[d])
print(hitdataset.shape)

In [ ]:
# predict the pIC50 values using the regression neural network model
uX = np.asarray(hitdataset[hitdataset.columns[1:(hitdataset.shape[1])]])
uY = model.predict(uX)

# sort the indexes of the predicted pIC50 values in descending order
w = np.squeeze(uY)
w = np.argsort(w)
w = w[::-1]

In [ ]:
# predict the pIC50 values using the regression neural network model
uX = np.asarray(hitdataset[hitdataset.columns[1:(hitdataset.shape[1])]])
uY = model.predict(uX)

# sort the predicted pIC50 values in descending order
z = np.squeeze(uY)
# z = np.sort(z)
# z = z[::-1]

In [ ]:
# Names of top 500 molecules
d = {}
for i in range(500):
    d[hitdataset.iloc[w[i]]['Name']] = z[w[i]]

In [ ]:
# save the list of top 500 names into a csv file
with open("top_500_molecules.csv", "w") as outfile:
    for name, pic in d.items():
        outfile.write(name)
        outfile.write('\t')
        outfile.write(str(pic))
        outfile.write("\n")